# TSA Chapter 6: Case Study - Granger Causality

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch6/TSA_ch6_case_granger/TSA_ch6_case_granger.ipynb)

Granger causality tests between interest rates.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Download interest rate data for cointegration case study
try:
    gs10 = web.DataReader('GS10', 'fred', '1990-01-01', '2024-01-01').resample('MS').last().dropna()
    gs2 = web.DataReader('GS2', 'fred', '1990-01-01', '2024-01-01').resample('MS').last().dropna()
    rates = pd.DataFrame({'10Y': gs10.iloc[:,0], '2Y': gs2.iloc[:,0]}).dropna()
except:
    np.random.seed(42); n = 408; dates = pd.date_range('1990-01-01', periods=n, freq='MS')
    rw = np.cumsum(np.random.normal(0, 0.1, n)) + 5
    rates = pd.DataFrame({'10Y': rw + np.random.normal(0, 0.2, n), '2Y': rw - 0.5 + np.random.normal(0, 0.15, n)}, index=dates)
print(f'Rates data: {len(rates)} obs')

In [ ]:
diff_rates = rates.diff().dropna()
fig, ax = plt.subplots(figsize=(8, 4))
results = []
for lag in range(1, 9):
    try:
        gc1 = grangercausalitytests(diff_rates[['10Y','2Y']], maxlag=lag, verbose=False)
        gc2 = grangercausalitytests(diff_rates[['2Y','10Y']], maxlag=lag, verbose=False)
        results.append({'lag': lag, '2Y->10Y': gc1[lag][0]['ssr_ftest'][1], '10Y->2Y': gc2[lag][0]['ssr_ftest'][1]})
    except: pass
if results:
    df = pd.DataFrame(results)
    ax.plot(df['lag'], df['2Y->10Y'], color=BLUE, lw=2, marker='o', label='2Y -> 10Y')
    ax.plot(df['lag'], df['10Y->2Y'], color=RED, lw=2, marker='s', label='10Y -> 2Y')
    ax.axhline(0.05, color=GRAY, ls='--', label='5% level')
ax.set_title('Granger Causality p-values', fontweight='bold'); ax.set_xlabel('Lag')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
plt.tight_layout(); save_chart(fig, 'ch6_case_granger'); plt.show()